In [1]:
import ipyvolume as ipv
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from ipywidgets import ColorPicker, VBox, \
    interact, interactive, fixed
import struct
import numpy as np
np.random.seed(12345) 

Firstly create sample data with make_blobs function which generates isotropic Gaussian blobs for clustering. Points are created in 3 dimiensions corresponding to n_features. 
Number of samples and centers can be choosen arbitrary.

In [2]:
X, _ = make_blobs(n_samples=500, n_features=3, centers=5)
X

array([[ 3.53607975,  4.61016309,  7.38895805],
       [ 9.19147934, -4.97549186, -6.09957173],
       [-9.3894151 , -6.75053368, -2.21182366],
       ...,
       [ 2.40129143,  5.00277387,  8.96946293],
       [ 1.77557344,  6.17219644, 10.38235873],
       [ 3.16239577,  5.97382819,  8.74878016]])

Let's create 3D scatter plot to see points.

In [29]:
fig = ipv.figure(height=600, width=600, layout={'width':'100%', 'height':'100%'})
scatter = ipv.scatter(*X.T, size=1, marker="sphere")
ipv.xyzlim(-10, 10)
display(fig)

Figure(camera=PerspectiveCamera(fov=46.0, position=(0.0, 0.0, 2.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.…

ipywidgets needs RGB tuple for coloring separate points,
so the hex number must be changed to tuple.

In [30]:
def hex_to_rgb(hex):
    hex = hex[1:]
    return struct.unpack('BBB', bytes.fromhex(hex))

This function is responsible for updating scatter points color with respect to ColorPickers.

In [31]:
def handle_cp_change(labels, **groups):
    group_ids = [int(g.split(' ')[1]) for g in groups.keys()]
    group_color = {k: hex_to_rgb(get_cp_value(cp)) for k, cp in zip(group_ids, groups.values())}
    colors = list(map(lambda x: group_color[x], labels))
    scatter.color = colors

def get_cp_value(cp):
    if type(cp) == ColorPicker:
        return cp.value
    else:
        return cp

Some colors are displayed in white. I chose few which displays properly.

In [32]:
avaliable_colors = {
    0: '#ff0000',
    1: '#00ff00',
    2: '#0000ff',
    3: '#ffff00',
    4: '#00ffff',
    5: '#ff00ff',
    6: '#000000'
}

Using K-means algorithm every point will be assigned to some cluster.
Number of clusters is parameter choosen by analyst.

To make it easier to investigate the best possible number it can be interactively changed.

Function bellow gets the n_clusters parameter and then calculates the predicted labels.
It also declares ColorPickers with assigned color which can be changed (colors out of avaliable colors will be white).

In [33]:
def color_scatter_with_kmeans(n_clusters):
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans = kmeans.fit(X)
    labels = kmeans.predict(X)
    color_pickers = {f'group {k}': ColorPicker(value=avaliable_colors[k%len(avaliable_colors)], description=f'group {k}') 
                     for k in range(n_clusters)}
    handle_cp_change(labels=list(labels), **color_pickers)
    return interact(handle_cp_change, labels=fixed(list(labels)) , **color_pickers)

Now wrap figure and widgets for interaction with VBox.

In [34]:
VBox([fig, interactive(color_scatter_with_kmeans, n_clusters=(1, 10, 1))])

You can change the number of clusters and see update of colors corresponding to the group. 

If colors make it hard to distinguish group you can change it with ColorPicker.